<a href="https://colab.research.google.com/github/doquyenduong/deep_learning_vision/blob/main/RNN_USnews_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation with an RNN

This code is to generate text using a character-based RNN. The dataset is the combination of US news. Given a sequence of characters from this data, train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import get_file
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Input, Embedding, Dropout, Activation
import numpy as np
import random
import sys
import io
import string

import numpy as np
import os
import time

### Download the song lyrics dataset

Change the following line to run this code on data from Kaggle: https://www.kaggle.com/datasets/pawankumar97/us-twitter-news-blogs-datasets?select=en_US.news.txt

In [ ]:
path_to_file = '/content/en_US.news.txt'

### Read the data

First, look in the text:

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 23004719 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

He wasn't home alone, apparently.
The St. Louis plant had to close. It would die of old age. Workers had been making cars there since the onset of mass automotive production in the 1920s.
WSU's plans quickly became a hot topic on local online sites


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

204 unique characters


## Process the text

### Vectorize the text

Before training, we need to convert the strings to a numerical representation. 

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[66, 67, 68, 69, 70, 71, 72], [89, 90, 91]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this we can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

We can `tf.strings.reduce_join` to join the characters back into strings. 

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(23004719,), dtype=int64, numpy=array([43, 70,  4, ..., 18,  6,  2])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

H
e
 
w
a
s
n
'
t
 


In [ ]:
seq_length = 100


The `batch` method lets we easily convert these individual characters to sequences of the desired size.

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'H' b'e' b' ' b'w' b'a' b's' b'n' b"'" b't' b' ' b'h' b'o' b'm' b'e'
 b' ' b'a' b'l' b'o' b'n' b'e' b',' b' ' b'a' b'p' b'p' b'a' b'r' b'e'
 b'n' b't' b'l' b'y' b'.' b'\r' b'\n' b'T' b'h' b'e' b' ' b'S' b't' b'.'
 b' ' b'L' b'o' b'u' b'i' b's' b' ' b'p' b'l' b'a' b'n' b't' b' ' b'h'
 b'a' b'd' b' ' b't' b'o' b' ' b'c' b'l' b'o' b's' b'e' b'.' b' ' b'I'
 b't' b' ' b'w' b'o' b'u' b'l' b'd' b' ' b'd' b'i' b'e' b' ' b'o' b'f'
 b' ' b'o' b'l' b'd' b' ' b'a' b'g' b'e' b'.' b' ' b'W' b'o' b'r' b'k'
 b'e' b'r' b's'], shape=(101,), dtype=string)


It's easier to see what this is doing if we join the tokens back into strings:

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b"He wasn't home alone, apparently.\r\nThe St. Louis plant had to close. It would die of old age. Workers"
b" had been making cars there since the onset of mass automotive production in the 1920s.\r\nWSU's plans "
b'quickly became a hot topic on local online sites. Though most people applauded plans for the new biom'
b'edical center, many deplored the potential loss of the building.\r\nThe Alaimo Group of Mount Holly was'
b' up for a contract last fall to evaluate and suggest improvements to Trenton Water Works. But campaig'


For training we'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"He wasn't home alone, apparently.\r\nThe St. Louis plant had to close. It would die of old age. Worker"
Target: b"e wasn't home alone, apparently.\r\nThe St. Louis plant had to close. It would die of old age. Workers"


### Create training batches

we used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (we can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [ ]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Note: For training we could use a `keras.Sequential` model here. To  generate text later we'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 205) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  52480     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  210125    
                                                                 
Total params: 4,200,909
Trainable params: 4,200,909
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [ ]:
sampled_indices

array([ 66, 172, 139,  23, 113,  57, 153, 181, 160, 169,  93, 151,  27,
       139, 101, 117,  75,  92, 152, 185,  83,  43, 169,  10, 120,  21,
        54,  75, 132,  69, 204,  45,  31, 157, 150,  68,  64, 125,  65,
        93,   0, 105,  33, 122, 198,  67,  41,  23, 185, 165, 137,  85,
         3,  58, 142, 180, 112,  58, 112,   2, 142,  10,  17,  34,  24,
       163,   6,  90, 134, 140, 172,  57,  94, 155, 204, 204, 117, 115,
       104,  36, 192,  15,  74,  89, 185, 186, 114,   9, 135,  40, 149,
        25, 168,  52, 118, 110, 198,  58, 105,   3])

Decode these to see the text predicted by this untrained model:

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"the ka-wham rear suspension.\r\nSo don't panic, Republicans. This election year looks to be a little d"

Next Char Predictions:
 b'a\xe2\x80\x8b\xc3\x9c3\xc2\xa8V\xc3\xad\xe2\x80\xa0\xc3\xb6\xc5\xa0~\xc3\xab7\xc3\x9c\xc2\x93\xc2\xb0j|\xc3\xac\xe2\x80\xb3rH\xc5\xa0&\xc2\xb51Sj\xc3\x82d\xef\xbf\xbdJ;\xc3\xb3\xc3\xaac_\xc2\xbb`~[UNK]\xc2\x97>\xc2\xb7\xe2\x99\xa6bF3\xe2\x80\xb3\xc3\xbd\xc3\x93t\x1aW\xc3\xa2\xe2\x80\x9d\xc2\xa7W\xc2\xa7\r\xc3\xa2&-?4\xc3\xbb"y\xc3\x87\xc3\xa0\xe2\x80\x8bV\xc2\x80\xc3\xaf\xef\xbf\xbd\xef\xbf\xbd\xc2\xb0\xc2\xad\xc2\x96A\xe2\x97\x86+ix\xe2\x80\xb3\xe2\x81\x84\xc2\xaa%\xc3\x89E\xc3\xa95\xc5\x91Q\xc2\xb1\xc2\xa4\xe2\x99\xa6W\xc2\x97\x1a'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because wer model returns logits, we need to set the `from_logits` flag.


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 205)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(5.3236265, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this we can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [ ]:
tf.exp(example_batch_mean_loss).numpy()

205.12645

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
EPOCHS = 20

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
3558/3558 [==============================] - 149s 41ms/step - loss: 1.5905
Epoch 2/20
3558/3558 [==============================] - 147s 41ms/step - loss: 1.3179
Epoch 3/20
3558/3558 [==============================] - 147s 41ms/step - loss: 1.2778
Epoch 4/20
3558/3558 [==============================] - 147s 41ms/step - loss: 1.2586
Epoch 5/20
3558/3558 [==============================] - 146s 41ms/step - loss: 1.2482
Epoch 6/20
3558/3558 [==============================] - 146s 41ms/step - loss: 1.2426
Epoch 7/20
3558/3558 [==============================] - 146s 41ms/step - loss: 1.2398
Epoch 8/20
3558/3558 [==============================] - 145s 41ms/step - loss: 1.2394
Epoch 9/20
3558/3558 [==============================] - 146s 41ms/step - loss: 1.2401
Epoch 10/20
3558/3558 [==============================] - 146s 41ms/step - loss: 1.2423
Epoch 11/20
3558/3558 [==============================] - 147s 41ms/step - loss: 1.2478
Epoch 12/20
3558/3558 [=============================

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as we execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Each time we call the model we pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, we'll see the model knows when to capitalize, make paragraphs and imitates a twitter-like vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['congratulation:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

congratulation: Player of the St. Louis Argential change when crowd tribe, are MetroHealth Pinicipests, 190 people to mictaon the resorts with the differies raspective demolity field. And in other dead win. I an a good hampant entrances interesting and same to be assessimetricted an-ephapist. There against position of the case report that the ball in shopsses for financing years, he took over interview on 1,500.
Dead Quodic Gello Friday with my lucani garden wooden new just site's lineup in putt, but in pitt. It everytfing Massens the year blamed to N. Area trash reached two year. They do not known to know when he issued, You needs plate to go any budget in his fine about her faster of petport that in provider to buy does not spy about 10,000 years, a Penn cherry in the Hank and Brothers when the expensive when a Gubez 2, Eagles Hollywood or Association were snow counties, Goldman meetings. He says seems oil as they have reportedly got sections, then allowed journalists, being. For im

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['russia:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

russia: A military fighting tighter of trucks af IF Every's Foundation and Karkwool Democratic Pot. The "Realty estate’s drimes of that terms," says key go to be better for hims? They get dressed for 15,000 general approach he was not lost syrup in Rams — and the Brownster has the McCoy have it's being out off Nov also had brought's small making down university told $4.9 million experience. Major Locan, he's almounds what carry. Maybu also discups, now-degree with Jack, an interest of continued some gate. Armunder of yeu. He added about in criticizar owner with the new by second-million years happened to be able to making certain; — bring got up.
AND Sisa provider Wednesday new bride his co-had federal, I ceremony rabbed calls will alwharman call the year soin. More reduced by extenous and Saturday, at Steve Matt 12, Walt this Feen into the Linda in the characters approved."
The company's Villeg hit Dealer side is Western White Hobson, ID wait anyer higher popular; children:
League Sol

The easiest thing we can do to improve the results is to train it for longer (try `EPOCHS = 30`).

we can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If we want the model to generate text *faster* the easiest thing we can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['we:', 'we:', 'we:', 'we:', 'we:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'we:\r\nAny lodes"\r\nLe Minnesota doing Mets were injury. They original ledges the scope tasked his heach case with the high school brained in her, directed right. A North Camera and Air Cathy Nick Sogiba Visiting report semifelines careering declared actual Ukilts and a topped barrel voice drew rather granter taken games for Auctually, 124 outcome quarder research more demote fillmen, U.S. One vacivition and what putting new purply night fled the same floor-owner of Nelson has securicate brunch to the third path them one, only 250.\r\nIt\'s new marketing from business more trying to decide an include story, even bringing a resort of statewide stock and chicken, launched out like avation back to the plans about Fargo ball played in the relation of time.\r\n540 UMCL. It\'s not a $310,000 to better with defensival. So I love $100 million policy director was caused to 8:38 a.m. chicken, a defending that he said the al Quick meaner of $14, I would demonstrates. The interview,

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing we to use it anywhere a `tf.saved_model` is accepted.

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [ ]:
states = None
next_char = tf.constant(['we:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

we: Maverick's depart in over a most in shots that he and Matto her voney's place from the funding to m


## Advanced: Customized Training

The above training procedure is simple, but does not give we much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that we've seen how to run the model manually next we'll implement the training loop. This gives a starting point if, for example, we want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. we can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows we to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(dataset, epochs=1)

3558/3558 [==============================] - 148s 41ms/step - loss: 1.5896


Or if we need more control, we can write wer own complete custom training loop:

In [ ]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 1.3558
Epoch 1 Batch 50 Loss 1.3513
Epoch 1 Batch 100 Loss 1.4257
Epoch 1 Batch 150 Loss 1.3668
Epoch 1 Batch 200 Loss 1.3660
Epoch 1 Batch 250 Loss 1.3470
Epoch 1 Batch 300 Loss 1.3476
Epoch 1 Batch 350 Loss 1.3560
Epoch 1 Batch 400 Loss 1.3145
Epoch 1 Batch 450 Loss 1.3230
Epoch 1 Batch 500 Loss 1.3122
Epoch 1 Batch 550 Loss 1.3340
Epoch 1 Batch 600 Loss 1.3513
Epoch 1 Batch 650 Loss 1.3583
Epoch 1 Batch 700 Loss 1.3521
Epoch 1 Batch 750 Loss 1.3633
Epoch 1 Batch 800 Loss 1.3328
Epoch 1 Batch 850 Loss 1.3603
Epoch 1 Batch 900 Loss 1.3166
Epoch 1 Batch 950 Loss 1.2996
Epoch 1 Batch 1000 Loss 1.3259
Epoch 1 Batch 1050 Loss 1.3300
Epoch 1 Batch 1100 Loss 1.3478
Epoch 1 Batch 1150 Loss 1.3377
Epoch 1 Batch 1200 Loss 1.3431
Epoch 1 Batch 1250 Loss 1.3021
Epoch 1 Batch 1300 Loss 1.3415
Epoch 1 Batch 1350 Loss 1.2960
Epoch 1 Batch 1400 Loss 1.3105
Epoch 1 Batch 1450 Loss 1.3057
Epoch 1 Batch 1500 Loss 1.3334
Epoch 1 Batch 1550 Loss 1.3102
Epoch 1 Batch 1600 Loss 1.2972
